Python风格纸牌游戏

下面是一个非常简单的例子，但它足以说明只实现了__getitem__ 和__len__特殊方法的强大之处：

In [2]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

使用collections.namedtuple构建一个简单的类来表示每张牌。从Python2.6起，namedtuple可以向数据库记录一样，只使用一组没有自定义方法的属性来构建对象的类

In [3]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [4]:
Card

__main__.Card

In [5]:
beer_card = Card('7', 'diamonds')

In [6]:
beer_card

Card(rank='7', suit='diamonds')

In [12]:
beer_card .rank

'7'

In [13]:
beer_card.suit

'diamonds'

虽然 FrenchDeck 隐式地继承了 object 类， 但功能却不是继承而来
的。 我们通过数据模型和一些合成来实现这些功能。 通过实现 __len__
和 __getitem__ 这两个特殊方法， FrenchDeck 就跟一个 Python 自有
的序列数据类型一样， 可以体现出 Python 的核心语言特性（例如迭代和
切片） 。 同时这个类还可以用于标准库中诸如
random.choice、 reversed 和 sorted 这些函数。 另外， 对合成的运
用使得 __len__ 和 __getitem__ 的具体实现可以代理给 self._cards
这个 Python 列表（即 list 对象） 。

In [14]:
deck = FrenchDeck()
len(deck)

52

In [15]:
deck[0]

Card(rank='2', suit='spades')

我们需要单独写一个方法用来随机抽取一张纸牌吗？ 没必要， Python 已
经内置了从一个序列中随机选出一个元素的函数 random.choice， 我
们直接把它用在这一摞纸牌实例上就好：

In [17]:
from random import choice
choice(deck)

Card(rank='6', suit='clubs')

那么排序呢？ 我们按照常规， 用点数来判定扑克牌的大小， 2 最小、 A
最大； 同时还要加上对花色的判定， 黑桃最大、 红桃次之、 方块再次、
梅花最小。 下面就是按照这个规则来给扑克牌排序的函数， 梅花 2 的大
小是 0， 黑桃 A 是 51：

In [36]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    #print(FrenchDeck.ranks, card.rank, len(suit_values), suit_values[card.suit])
    return rank_value * len(suit_values) + suit_values[card.suit]

In [37]:
spades_high(beer_card)

21

有了 spades_high 函数， 就能对这摞牌进行升序排序了：

In [38]:
for card in sorted(deck, key=spades_high): # doctest: +ELLIPSIS
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

首先明确一点， 特殊方法的存在是为了被 Python 解释器调用的， 你自己
并不需要调用它们。 也就是说没有 my_object.__len__() 这种写法，
而应该使用 len(my_object)。 在执行 len(my_object) 的时候， 如果
my_object 是一个自定义类的对象， 那么 Python 会自己去调用其中由
你实现的 __len__ 方法。

### 模拟数值类型

利用特殊方法， 可以让自定义对象通过加号“+”（或是别的运算符） 进
行运算。

In [46]:
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    #Python 有一个内置的函数叫 repr， 它能把一个对象用字符串的形式表达出来以便辨认， 这就是“字符串表示形式”
    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        #print(other.x,other.y)
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

实现一个二维向量（vector） 类， 这里的向量就是欧几里得几何
中常用的概念,向量加法

In [47]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)

In [48]:
v1 + v2

2 1


Vector(4, 5)

abs 是一个内置函数， 如果输入是整数或者浮点数， 它返回的是输入值
的绝对值； 如果输入是复数（complex number） ， 那么返回这个复数的
模。

In [49]:
v = Vector(3, 4)
abs(v)

5.0

我们还可以利用 * 运算符来实现向量的标量乘法（即向量与数的乘法，
得到的结果向量的方向与原向量一致 ， 模变大

In [50]:
v * 3

Vector(9, 12)

对 __bool__ 的实现很简单， 如果一个向量的模是 0， 那么就返回
False， 其他情况则返回 True。 因为 __bool__ 函数的返回类型应该是
布尔型， 所以我们通过 bool(abs(self)) 把模值变成了布尔值。

In [51]:
bool(Vector(3, 4))

True